The next level of recommender systems is to utilize more traditional machine learning techniques.

We will also review the literature about matrix factorization methods. The next part will cover an example of two technqiues.

Lets load our data in first.

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline


# pass in column names for each CSV
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,
                    encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,
                      encoding='latin-1')

# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('ml-100k/u.item', sep='|', names=m_cols, usecols=range(5),
                     encoding='latin-1')

# create one merged DataFrame
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)

In [5]:
n_users = lens['user_id'].max()
n_items = lens['movie_id'].max()

# split to train test

from sklearn import cross_validation as cv
df_train, df_test = cv.train_test_split(lens, test_size=0.2)

# generate train test matrix
def generate_train_matrix(df):
    train_matrix = np.zeros((n_users, n_items))
    for i in df.itertuples():
        # assume that row id is always an integer from 1 to n
        row_i = i.user_id - 1
        # same for column id
        col_i = i.movie_id - 1
        train_matrix[row_i, col_i] = i.rating
    return train_matrix
train_matrix = generate_train_matrix(df_train)
print train_matrix.shape

(943, 1682)


# The basics

Model based methods utilize traditional machine learning algorithms to predict the rating per item for a user. 

Some key advantages:
- Space-efficiency: store the model per item
- Training speed and prediction speed: much faster
- avoid overfitting: Summarizes a model per item


# 1. Decision/Regression Trees 
This will not cover the details of decision trees as it is not that special from the vanilla formulation.

Given rating matrix $R$ which is a $m \times n$ for $m$ users and $n$ items, build a ML model for each item. Therefore, build $n$ models. The features for model $i$ is the interactions a user has with all the other items. Each user is a data point used to train the item model. 

Often, the matrix is very sparse. For example, some items might only have one user interact with it. If using a method such as decision trees, the split could get stuck at a single item. To minimize these, feature reduction methods (ie. PCA or SVD) can be used. Method reduction transforms the input vector into a new vector space that is of lower dimensions yet contains most of the information.

In [1]:
from sklearn.tree import DecisionTreeClassifier
# use a hash to store models
# key : item ID
# Value: a decision model
models = {}
for i in xrange(n_items):
    

# 2. Association Rules (Apiori Algorithm)

Assume unary ratings, $r = \{0,1\}$, for now. Association rules mine transactions $T = \{T_1,...,T_m\}$ (one transaction per user) where $T_i$ is a subset of all $n$ items $I$. Associate mining looks to determine sets of items, **itemset**, in the entire dataset that are correlated/associated with each other. There is some important terminology for this algorithm type. The _support_ and the _confidence_.
- _support_ of itemset $X \subseteq I$ is the fraction of transactions that contain the subset $X$ over all all transactions in $T$. 
$$supp(X)=\sum_{i=1}^{m} \frac{I(X \subseteq T_i)}{N_T}$$

The support is similar to the frequency an itemset appears in a set of transactions.

- _confidence_ of a rule $X \implies Y$ is the conditional probability that a transaction, $T_i$, contains $Y$ given that it contains $X$. IN other words, confidence is the support of $X \cup Y$ divided by the support of $X$

$$conf(X \implies Y) = \frac{supp(X \cup Y)}{supp(X)} $$
For example, let a database $I$ contain all items ${0,...,}$ with there are _5_ transactions. The transactions are $T =\{0,1,2,3\},\{0,1,3\},\{1,2\}, \{2,4\},\{0,1,2,5\}$

Let $X={1,2}$. The $supp(X)$ is $\frac{1 + 0 + 1 + 0 + 1}{5} =0.6$. 

Let $Y={3}$. The $conf(X \implies Y)$ is $\frac{1/5}{0.6}=0.333$

### The Apiori Algorithm

The algorithm has two parameters that confirm $X \implies Y$ as an associate rule where:

- The support of $X \cup Y$ has a **minimum support** of $s$
- The confidence of $X \implies Y$ is at least **minimum confidence** c

The algorithm uses two phases.

1. The first phase computes all the permutations of itemsets such that the minimum support is satisfied. Only those that exceed the minimum support are kept.
2. The next phase looks at all the associate rules that satisfy the minimum confidence. So for permutations of itemset $Z$, all associate rules of $X \implies Z - X$ are computed. Those that satisfy the minimum confidence are associate rules.

This process is computationally heavy. Methods of efficient computation are discussed seperately. 

#### Application to Recommender Systems


For a user with current item ratings, we can predict future items. For current itemset $X_i$ for user $i$, the minimum confidence of having all possible items are computed. An association rule is _fired_ if an itemset that is a subset of all user rated items is an associate rule as determined by the apiori algorithm. Future items are computed alongisde the current itemset $X_i$. The items are then ranked from highest to lowest confidence and the top-k results are selected as the recommended items. This method is suitable for unary ratings. 

For non-binary ratings, we can expand the item matrix to include discrete ratings per item. For example, if the rating system is $r=\{like,neutral,dislike\}$ for items $\{1,2\}$, we include entries to be $I=\{1 \ like, 1 \ dislike, 2 \ like, 2 \ dislike \}$. The neutral would be a 0 in both columns per items. So association rules could be

$$ \{Bread \ Like, Milk \ Dislike \} \implies \{Butter \ like \}$$

One problem that could occur is when conflicting ratings for the same item appears in the top-k results. One solution is to aggregate all the high-confident scores for a particular item. For example, if 3 different associate rules for a user are fired above the minimum confidence, the average confidence of the 3 associate rules can be used to compute an aggregate confidence. The items can then be reweighted.

Associate rules can be computed either item-wise or user-wise. Transactions are itemsets that occur frequently. User-wise associate rules use the item ratings per user as entries where each row of the rating matrix is a data entry. Item-wise associate rules uses user rating per item to predict what the item will be rated. Per item has a list of psuedo-users that have ratings for it. For a new user and an item, the psuedo users can be used to find what types of users are associated with the item and have similar behaviour.

# 3. Naive Bayes 
This is a generaliztion of the classification algorithm used by Naive Bayes for classification. Assume that each ordinal rating, $v_s$, of an item is a distinct class. The distcrete ratings are the ordinal ratings an user could have per item. The ratings range from $\{v_1,..., v_l\}$. In other words $ s \ \epsilon \{1,...,l\}$

For a user $u$, their rated items is written as $I_u$. For a new item $j$, the predicted rating is the ordinal classes the maximizes the probability the item is rated a particular class given the likelihood of the users previous ratings. In other words:

$$ P(r_{uj}=v_s|\text{Observed ratings in } I_u) = \frac
    {P(r_{uj}=v_s) \bullet P(\text{Observed ratings in } I_u | r_{uj} =v_s)}
    {P(\text{Observed ratings in } I_u )}
$$

Where the prior is the fraction of users who rated item $j$ with the ordinal rating of $v_s$ over all users who rated item $j$. The likelihood probability is computed as the fraction of users that have specified the rating of $r_{uk}$ for the kth item given they have rating for item $j=v_s$. In othewords, how common the other ratings of user $u$ are among all other users' ratings given they rated item $j$.

$$ P(\text{Observed ratings in } I_u | r_{uj} =v_s) = \prod_{k \epsilon I_u} P(r_{uk}|r_{uj}=v_s)$$

There are two approaches to utilizing this method for item recommendations. For systems that have low variations in ratings (ie binary ratings), the vanilla approach is acceptable. If a system has many possible classes, one approach is to take the weighted average of the ordinal ratings (weighted by the probabilit).

$$ \hat{r_{uj}} = \sum_{s=1}^{l} \frac
    {v_s \bullet P(r_{uj}=v_s|\text{Observed ratings in } I_u)}
    {P(r_{uj}=v_s|\text{Observed ratings in } I_u)} 
$$

## Reducing Overfitting 

To reduce the case where data is sparse (heavy skew) we can use laplacian smoothing to re-estimate the prior.

$$ P(r_{uj} = v_s) \frac{q_s + \alpha}{\sum_{t=1}^{l} q_t + l \bullet \alpha} $$

Where $q_1...q_l$ is the number of users that have specified ratings for $v_1...v_l$ for the $j$th item with the Laplcian smoothing parameter $\alpha$.

# 4. Black Box Approach
The approach here is to use any regression or classification model. Similar to the decision tree, we generate a predictive model per item (for item-wise models). So we end up with $n$ models. Because traditional models need the data matrix to be fileld (ie. every item needs some rating), the empty cells (items we are TRYING to predict) need to be initalized. To reduce bias, we have to shift all current ratings by the mean, so subtract each rating by the row-mean (mean of the user rating). This accounts for the bias that each user could introduce. We can then initalize each empty cell with the mean of the user.

The empty cells can then be predicted by the item-model using the other item ratings as the input feature. To create a roust model, we can iterate on the predicted model through a 2 step iterative algorithm . For algorithm $A$, we assume two phases:

1. Use algorithm $A$ to estimate the missing entries for each column by setting it as the response variate. Use the default values for other missing cells.

2. Update all the missing entries based on the algorithim $A$. Repeat the training process with the updated cells, giving a boosted model.

Can use either classification or regression models. Classification can rank items by comparing likelihood scores. This allows for _any_ vanilla machine learning alogrithm for this application.

# Matrix Factorization